In [1]:
import cv2
import face_recognition
from datetime import datetime
import pandas as pd

In [18]:
def compare_faces(employee_image_path, camera_feed):
    # Load the employee's image for comparison
    employee_image = face_recognition.load_image_file(employee_image_path)
    employee_encoding = face_recognition.face_encodings(employee_image)[0]

    # Convert the camera feed to RGB format (face_recognition uses RGB)
    camera_rgb = cv2.cvtColor(camera_feed, cv2.COLOR_BGR2RGB)

    # Detect faces in the camera feed
    face_locations = face_recognition.face_locations(camera_rgb)
    face_encodings = face_recognition.face_encodings(camera_rgb, face_locations)

    # Compare the detected face with each employee's face using face_recognition library
    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces([employee_encoding], face_encoding)

    if True in matches:
        for _, employee_info in employee_data.iterrows():
            if employee_info['ImagePath'] == employee_image_path:
                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                # Check if the employee data is already present in the CSV file
                try:
                    df = pd.read_csv("employee_log.csv")
                except FileNotFoundError:
                    df = pd.DataFrame(columns=["EmployeeID", "Name", "Position", "Department", "Date and Time", "Action"])

                # Check if the employee is already in the CSV log
                if employee_info['EmployeeID'] in df["EmployeeID"].values:
                    last_action = df[df["EmployeeID"] == employee_info['EmployeeID']].iloc[-1]["Action"]
                    if last_action == "Entered":
                        action = "Exited"
                    else:
                        action = "Entered"
                else:
                    action = "Entered"

                # Append the new entry to the CSV file
                new_entry = pd.DataFrame([[employee_info['EmployeeID'], employee_info['Name'],
                                           employee_info['Position'], employee_info['Department'],
                                           current_time, action]],
                                         columns=["EmployeeID", "Name", "Position", "Department", "Date and Time", "Action"])
                df = pd.concat([df, new_entry], ignore_index=True)

                # Save the updated DataFrame to the CSV file
                df.to_csv("employee_log.csv", index=False)

                # Print the employee information and action
                print(f"Employee ID: {employee_info['EmployeeID']}")
                print(f"Name: {employee_info['Name']}")
                print(f"Position: {employee_info['Position']}")
                print(f"Department: {employee_info['Department']}")
                print(f"Date and Time: {current_time}")
                print(f"Action: {action}")

                return True

    return False

def delete_data():
    try:
        df = pd.read_csv("employee_log.csv")
        if not df.empty:
            df.drop(df.index, inplace=True)
            df.to_csv("employee_log.csv", index=False)
            print("Data in the log file has been deleted.")
        else:
            print("Log file is already empty.")
    except FileNotFoundError:
        print("Log file not found.")

if __name__ == "__main__":
    # Read employee information from the CSV file
    employee_data = pd.read_csv("employee_info.csv")

    camera = cv2.VideoCapture(0)  # Access the laptop camera

    ret, frame = camera.read()
    if ret:
        is_employee_detected = False
        for _, employee_info in employee_data.iterrows():
            if compare_faces(employee_info['ImagePath'], frame):
                is_employee_detected = True
                break

        if not is_employee_detected:
            print("Not an employee.")

    camera.release()
    cv2.destroyAllWindows()


Employee ID: employee_id_1
Name: Sina Dehghanian
Position: Data Scientist
Department: Data Science
Date and Time: 2023-07-26 16:14:01
Action: Exited


In [11]:
# Call the delete_data() function to remove data from the CSV log file
delete_data()

Data in the log file has been deleted.
